In [ ]:
from datasets import load_dataset
# from accelerate import Accelerator
import argparse
import torch
import os
import json
from itertools import chain
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    DataCollatorWithPadding,
    default_data_collator,
    AutoTokenizer,
    AutoModelForMultipleChoice,
    AutoModelForQuestionAnswering
)
from accelerate import Accelerator
from utils_qa import postprocess_qa_predictions


os.environ["CUDA_VISIBLE_DEVICES"]="0"
# accelerator = Accelerator()
# accelerator.wait_for_everyone()
# prompt = "How the weather today?"
# candidate2 = "My name is Pekora."
# candidate1 = "It's sunny outside."

model_path = "/nfs/nas-6.1/whlin/ADL/2023_ADL_HW1/checkpoint/paragraph_select/xlnet_base"


# -------------------------- prepare dataset
tokenizer_path = "/nfs/nas-6.1/whlin/ADL/2023_ADL_HW1/checkpoint/paragraph_select/xlnet_base"

# load raw dataset
raw_datasets = load_dataset("json", data_files={"test": "./data/valid.json"})

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# load context data
with open("./data/context.json") as f:
    context_data = json.load(f)

def turn_to_swag_format(example):
    data = dict()
    data['label'] = [example['paragraphs'][i].index(answer_id) for i, answer_id in enumerate(example['relevant'])]
    # there are four possible context for an question
    for i in range(4):
        data['sentence{}'.format(str(i))] = [context_data[paragraph_ids[i]] for paragraph_ids in example['paragraphs']]

    return data

ending_names = [f"sentence{i}" for i in range(4)]
context_name = "question"

def preprocess_function(examples):
        # queston * 4
        first_sentences = [[context] * 4 for context in examples[context_name]]
        second_sentences = [
            [f"{examples[end][i]}" for end in ending_names] for i in range(len(examples[context_name]))
        ]
        # Flatten out
        first_sentences = list(chain(*first_sentences))
        second_sentences = list(chain(*second_sentences))

        # Tokenize
        tokenized_examples = tokenizer(
            first_sentences,
            second_sentences,
            max_length=512,
            padding="max_length",
            truncation=True,
        )
        # Un-flatten
        tokenized_inputs = {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
        return tokenized_inputs

swag_format_datasets = raw_datasets.map(turn_to_swag_format, batched = True, remove_columns=raw_datasets['test'].column_names.remove("question"))
processed_datasets = swag_format_datasets.map(
    preprocess_function, batched=True, remove_columns=swag_format_datasets['test'].column_names
)


test_dataset = processed_datasets['test'].select(range(32))
test_dataloader = DataLoader(test_dataset, collate_fn = default_data_collator, batch_size=16)


# ----------------- prepare model --------------
# load model
model = AutoModelForMultipleChoice.from_pretrained(model_path)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu");
model.to(device)

In [ ]:
model.eval()
all_prediction = list()
for batch in test_dataloader:    
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions = predictions.detach().cpu().clone().numpy()
    all_prediction.extend(predictions)

print(all_prediction)

In [ ]:
print(swag_format_datasets['test'].select(range(32))['label'])

In [ ]:
ds = raw_datasets['test'].select(range(32)).add_column("label2", all_prediction)

In [ ]:
print(ds)

In [ ]:
ds.column_names

In [ ]:
print(ds['label2'])

In [ ]:
print(raw_datasets['test'].column_names)
print(ds.column_names)

In [ ]:
remove_columns = raw_datasets['test'].column_names
remove_columns = ['answer']
print(remove_columns)

In [ ]:
def postprocess(examples):
    return {'relevant': [examples['paragraphs'][idx][label] for idx, label in enumerate(examples['label2'])]}

output_dataset = ds.map(postprocess, batched = True, remove_columns= ['paragraphs', 'relevant', 'answer', 'label2'])
# output_dataset.to_json("./data/paragraph_select_test_result.json", force_ascii=False) 

In [ ]:
print(output_dataset[0])

# Question Answering Part

In [ ]:
"""
Post-processing utilities for question answering.
"""
import collections
import json
import logging
import os
from typing import Optional, Tuple

import numpy as np
from tqdm.auto import tqdm


logger = logging.getLogger(__name__)


def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    # if output_dir is not None:
    #     if not os.path.isdir(output_dir):
    #         raise EnvironmentError(f"{output_dir} is not a directory.")

    #     prediction_file = os.path.join(
    #         output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
    #     )
    #     nbest_file = os.path.join(
    #         output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
    #     )
    #     if version_2_with_negative:
    #         null_odds_file = os.path.join(
    #             output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
    #         )

    #     logger.info(f"Saving predictions to {prediction_file}.")
    #     with open(prediction_file, "w") as writer:
    #         writer.write(json.dumps(all_predictions, indent=4) + "\n")
    #     logger.info(f"Saving nbest_preds to {nbest_file}.")
    #     with open(nbest_file, "w") as writer:
    #         writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
    #     if version_2_with_negative:
    #         logger.info(f"Saving null_odds to {null_odds_file}.")
    #         with open(null_odds_file, "w") as writer:
    #             writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [ ]:

# Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
    """
    Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor

    Args:
        start_or_end_logits(:obj:`tensor`):
            This is the output predictions of the model. We can only enter either start or end logits.
        eval_dataset: Evaluation dataset
        max_len(:obj:`int`):
            The maximum length of the output tensor. ( See the model.eval() part for more details )
    """

    step = 0
    # create a numpy array and fill it with -100.
    logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
    # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather_for_metrics
    for i, output_logit in enumerate(start_or_end_logits):  # populate columns
        # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
        # And after every iteration we have to change the step

        batch_size = output_logit.shape[0]
        cols = output_logit.shape[1]

        if step + batch_size < len(dataset):
            logits_concat[step : step + batch_size, :cols] = output_logit
        else:
            logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

        step += batch_size

    return logits_concat


def question_answering(context_data, raw_datasets):
    accelerator = Accelerator()
    accelerator.wait_for_everyone()
    model_path = "/nfs/nas-6.1/whlin/ADL/2023_ADL_HW1/checkpoint/qa/roberta_large/"

    # prepare dataset
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    pad_on_right = tokenizer.padding_side == "right"
    question_column_name = "question" 
    context_column_name = "context" 
    max_seq_length = min(512, tokenizer.model_max_length)

    def turn_context_id_to_context(example):
        data = dict()
        data['context'] = context_data[example['relevant']]  
        return data
    
    raw_datasets = raw_datasets.map(turn_context_id_to_context, remove_columns=['relevant'])
    
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if False else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples
    
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=False,
            n_best_size=20,
            max_answer_length=128,
            null_score_diff_threshold=0.0,
            prefix=stage,
        )
        # Format the result to the format the metric expects.
        # if args.version_2_with_negative:
        #     formatted_predictions = [
        #         {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        #     ]
        # else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

        return formatted_predictions

    # DataLoaders creation:
    if False:
        # If padding was already done ot max length, we use the default data collator that will just convert everything
        # to tensors.
        data_collator = default_data_collator
    else:
        # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
        # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
        # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))


    column_names = raw_datasets.column_names
    predict_examples = raw_datasets
    
    with accelerator.main_process_first():
        predict_dataset = predict_examples.map(
            prepare_validation_features,
            batched=True,
            remove_columns=column_names,
            load_from_cache_file= True,
            desc="Running tokenizer on prediction dataset",
        )


    predict_dataset_for_model = predict_dataset.remove_columns(["example_id", "offset_mapping"])
    predict_dataloader = DataLoader(
        predict_dataset_for_model, collate_fn=data_collator, batch_size=8
    )

    # ------------------- do predict ----------------------
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    model, predict_dataloader = accelerator.prepare(model, predict_dataloader)
    
    all_start_logits = []
    all_end_logits = []
    model.eval()
    for step, batch in enumerate(predict_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

            if not False:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather_for_metrics(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather_for_metrics(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)
    prediction = post_processing_function(predict_examples, predict_dataset, outputs_numpy)
    return prediction


In [ ]:

with open("./data/context.json") as f:
    context_data = json.load(f)

final_prediction = question_answering(context_data, output_dataset)

In [ ]:
final_prediction

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_records(final_prediction)

In [ ]:
df.to_csv("./data/test_qa_result.csv", index=False)

: 